# What is Stabilizer Formalism and why do I need it? 

Quantum error correction is an important domain in the universe of quantum computing. There are several quantum error correction codes to mitigate and combat the errors. The **stabilizer formalism** is a framework for the quantum error correction codes, please observe that it is not a new error correction code on its own by rather a new representation to work with. There are also many error correction codes which doesn't fall under the umberalla of stabilizer formalism. We will not discuss those in this article.

Simplest example of error correcting code is the repeatition code. Before we dive deep into the stabilizer formalism, let us draw some inspiration by examining the repeation code from a new angle. 

<!-- The **stabilizer formalism** is a framework used in quantum error correction. It enables the characterization of quantum states using stabilizer groups, which are subgroups of the Pauli group. This notebook demonstrates the stabilizer formalism through examples implemented in Python with PennyLane. -->

## Repeatition code

As you know, the codespace of the repeatition code is defined by,

$$ \ket{\psi} = \alpha \ket{000} + \beta \ket{111} $$

Imagine if there is a bit flip error an error on the codespace it leads to an error space given by,

$$ X_2 \ket{\psi}_e = \alpha \ket{010} + \beta \ket{101}  $$

How is this error detected? Unlike classical case, no-cloning theorem prevents us from measuring the qubits to learn about the error. Before, worrying about how to detect an error, first let us understand what kinds of errors are possible. Observe the fact that the only detectable error is a single bit flip error on any of the qubits, therefore all we need to check is if any ONE of the bits flipped. In order to do this, we can build a circuit which checks if the first and second qubits are same or different and similary if second and third qubits are same or different. Okay how do we build one? Let us use PennyLane! 

It is divided into 3 steps. First step is to prepare the state, as from above we can use any alpha and beta to create the state but let us use the simplest one, that is using 1/sqrt 2 for both alpha and beta. The code block below demonstrates that


In [ ]:
import pennylane as qml
from pennylane import numpy as np

# Define the device
dev = qml.device("default.qubit", wires=3)

# Define the quantum circuit for encoding the state
@qml.qnode(dev)
def encode_circuit():
    # Apply the Hadamard gate to qubit 0 (creating a superposition between |0> and |1>)
    qml.Hadamard(wires=0)
    
    # Encode the state into 3 qubits (using repetition code)
    qml.CNOT(wires=[0, 1])  # CNOT from qubit 0 to qubit 1
    qml.CNOT(wires=[0, 2])  # CNOT from qubit 0 to qubit 2
    
    return qml.state()  # Return the full quantum state

# Run the encoding circuit
encoded_state = encode_circuit()
print("Encoded State:", encoded_state)


$$
\text{The output vector represents the quantum state } \frac{1}{\sqrt{2}}(|000\rangle + |111\rangle), 
$$
$$
\text{where the amplitudes for } |000\rangle \text{ and } |111\rangle \text{ are } \frac{1}{\sqrt{2}},
$$
$$
\text{and the amplitudes for all other states are 0.}
$$
Now let us introduce the X error:

In [ ]:
# Define the quantum circuit to introduce X error on qubit 2
@qml.qnode(dev)
def apply_x_error():
    # Apply X error on qubit 2 (flip the state of qubit 2)
    qml.PauliX(wires=2)
    
    return qml.state()  # Return the state after the error is applied

# Run the circuit with the X error on qubit 2
state_with_error = apply_x_error()
print("State with X error on qubit 2:", state_with_error)


Now let us build the circuit to do the detect the error.

In [5]:
# Define the quantum circuit to detect the error on the qubits
@qml.qnode(dev)
def detect_error():
    # Decode the state (repetition code)
    qml.CNOT(wires=[1, 0])  # CNOT from qubit 1 to qubit 0
    qml.CNOT(wires=[2, 0])  # CNOT from qubit 2 to qubit 0
    
    # Measure the state of all qubits
    return qml.probs(wires=[0, 1, 2])  # Probabilities of all qubits' states

# Run the circuit to detect error
probabilities = detect_error()
print("Measurement probabilities for all qubits:", probabilities)


Measurement probabilities for all qubits: [1. 0. 0. 0. 0. 0. 0. 0.]


Technically, what we did in the detection part is called the parity check. It can be done by just measuring the the operators z1z2 and z2z3. It is preciesely these opertors that are called stabilizers. What happens is that these operators break the codespace into several subspaces as shown in figure...

Now the question is that given these operators, will they form a valid quantum error correcting code? We came half way starting from repittion code to stabilizers, now let us complete the discussion by starting form the other half.

As a small exercise convince yourself that a phase flip error cannot be detected using the above code. 

# Stabilzer formalism

A few definitions first:

Pauli group:

Stabilizer group:

Any abilean subset of the pauli group with 